# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

_Answers: Bill Ott_


For Project 2, you will complete a series of exercises exploring __[order data from Chipotle](https://github.com/TheUpshot/chipotle)__, compliments of The New York Times' "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

In [1]:
#importing needed libraries
import csv
import pandas as pd

## Basic Level

### Part 1: Read in the file with csv.reader() and store it in an object called file_nested_list.

In [2]:
#find the tsv, open it
with open('./chipotle.tsv') as tsv:
    
    #emtpy list where the data will be stored
    chipotle_raw = []
    
    #create the file_nest_list object using csv reader using tab as delimiter
    file_nested_list = csv.reader(tsv, delimiter="\t")
    
    #iterate through each row in the object and append to our empty list
    for row in file_nested_list:
        chipotle_raw.append(row)

### Part 2: Separate file_nested_list into the header and the data.

In [3]:
#create variables for both the header and the data, both of which are stored in our list
chipotle_header = chipotle_raw[0]
chipotle_data = chipotle_raw[1:]

In [4]:
#create a dataframe from the two new variables (to more easily perform analyses below)
chipotle_df = pd.DataFrame(chipotle_data, columns=chipotle_header)

#clean up and type cast the numeric columns
chipotle_df['item_price'] = chipotle_df['item_price'].str.replace('$','')
chipotle_df['item_price'] = chipotle_df['item_price'].astype(float)
chipotle_df['quantity'] = chipotle_df['quantity'].astype(int)

## Intermediate Level

### Part 3: Calculate the average price of an order.

In [5]:
#subset data to just order_id & item_price, groupby the order_id, sum the item_price
order_total = chipotle_df.groupby('order_id')['item_price'].agg(['sum']).rename(columns={'sum':'order_total'})

#calculate the average order_total
order_total.mean()

order_total    18.811429
dtype: float64

### Part 4: Create a list (or set) named unique_sodas containing all of unique sodas and soft drinks that Chipotle sells.

In [6]:
#subset data to just item_name that are soda
soda_series = chipotle_df[(chipotle_df['item_name'] == 'Canned Soda')
            |(chipotle_df['item_name'] == 'Canned Soft Drink')
            |(chipotle_df['item_name'] == '6 Pack Soft Drink')]

#create list of unique choice_description (aka all the sodas)
sodas = list(soda_series['choice_description'].unique())

#clean up the names in the list for readability
sodas = [soda.replace('[','') for soda in sodas]
sodas = [soda.replace(']','') for soda in sodas]

#sodas!
sodas

['Sprite',
 'Dr. Pepper',
 'Mountain Dew',
 'Diet Dr. Pepper',
 'Coca Cola',
 'Diet Coke',
 'Coke',
 'Lemonade',
 'Nestea']

## Advanced Level

### Part 5: Calculate the average number of toppings per burrito.

In [7]:
#create a new df from subset containing only item_name that have 'Burrito'
burritos_df = pd.DataFrame(chipotle_df[chipotle_df['item_name'].str.contains('Burrito')])

#create a new column for the ingredients in the burritos + clean up the strings
burritos_df['ingredients'] = burritos_df['choice_description'].str.replace('[','')
burritos_df['ingredients'] = burritos_df['ingredients'].str.replace(']','')
burritos_df['ingredients'] = burritos_df['ingredients'].str.replace(', ',',')

#change ingredients from a string to a list of strings containing each ingredient
burritos_df['ingredients'] = burritos_df['ingredients'].str.split(",",n=-1,expand = False)

#add a new column for the number of ingredients (just in case we want to use this later)
burritos_df['ingredient_count'] = burritos_df['ingredients'].str.len()

#calculate the average toppings per burrito
burritos_df['ingredient_count'].mean()

5.395051194539249

### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

In [8]:
#create a new df from subset containing only item_name that have 'Chips'
chips_df = pd.DataFrame(chipotle_df[chipotle_df['item_name'].str.contains('Chips')])

#clean up string values in item_name
'''
NOTE: doing this will naturally affect the sum of 'quantity' (but it's more accurate, so w/e)
'''
chips_df['item_name'] = chips_df['item_name'].str.replace('Chili Corn','Chili-Corn')
chips_df['item_name'] = chips_df['item_name'].str.replace('Tomatillo ','Tomatillo-')

#change the quantity type from str to int
chips_df['quantity'] = chips_df['quantity'].astype(int)

#create a dictionary with the chip item_name and quantity
chip_orders = chips_df.groupby('item_name')['quantity'].sum().sort_values(ascending=False).to_dict()
chip_orders

{'Chips and Guacamole': 506,
 'Chips': 230,
 'Chips and Fresh Tomato Salsa': 130,
 'Side of Chips': 110,
 'Chips and Tomatillo-Green Chili Salsa': 78,
 'Chips and Tomatillo-Red Chili Salsa': 75,
 'Chips and Roasted Chili-Corn Salsa': 41,
 'Chips and Mild Fresh Tomato Salsa': 1}

## Bonus: Craft a problem statement about this data that interests you, and then answer it!

### What percentage of mains (not sides or drinks) are ordered with guacamole?

In [9]:
#define a function that categorizes each item so we can distinguish the mains
'''
to write the conditions in this function, I had to look at all of the unique values for 'item_name' 
and decide what patterns to utilize.
'''
def item_cat(df):
    if 'Burrito' in df['item_name']:
        cat = 'main'
    elif 'Taco' in df['item_name']:
        cat = 'main'
    elif 'Salad' in df['item_name']:
        cat = 'main'
    elif 'Bowl' in df['item_name']:
        cat = 'main'
    elif 'Chips' in df['item_name']:
        cat = 'side'
    else:
        cat = 'drink'
    return cat

#apply that function to the df and add the return value as a new item category column
chipotle_df['item_cat'] = chipotle_df.apply(item_cat,axis=1)

#create a new ingredients column, just like we did with the burrito_df, but leaving it a string
chipotle_df['ingredients'] = chipotle_df['choice_description'].str.replace('[','')
chipotle_df['ingredients'] = chipotle_df['ingredients'].str.replace(']','')
chipotle_df['ingredients'] = chipotle_df['ingredients'].str.replace(', ',',')

#add a new column to see if guacamole is present in order
chipotle_df.loc[(chipotle_df['ingredients'].str.contains('Guacamole')),'guac_present'] = 1
chipotle_df['guac_present'].fillna(0, inplace=True)

#create subset that only looks at mains and mains with guac
mains = chipotle_df[chipotle_df['item_cat'] == 'main']
mains_guac_present = mains[mains['guac_present'] == 1]

#quantify the base components, calculate the percentage
num_mains = mains['quantity'].sum()
num_mains_guac_present = mains_guac_present['quantity'].sum()
guac_percentage = (num_mains_guac_present/num_mains)

#finish this off with a neatly written print statement
print("Guac is present "+"{:.0%} of mains ordered.".format(guac_percentage))

Guac is present 36% of mains ordered.
